In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/kaggle/input/transformersscript/')

In [7]:
import pandas as pd
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
import numpy as np
# from timeseries_preprocessing import timeseries_dataset_from_array
from tensorflow.keras.preprocessing import timeseries_dataset_from_array

file_path = '/Users/mobin/Documents/quant finance Interview/My interviews/Kaggle/'
df1 = pd.read_csv(file_path+ "household_power_consumption.txt",delimiter=";")
df1["datetime"] = df1[["Date","Time"]].apply(lambda x: x[0]+" "+x[1],axis=1)
df1["datetime"] = pd.to_datetime(df1["datetime"], infer_datetime_format=True)
df1 = df1.drop(["Date","Time"],axis=1)
df1.index = df1["datetime"]
df1 = df1.drop("datetime",axis=1)
df1 = df1.apply(lambda x: pd.to_numeric(x, errors='coerce'))

/Users/mobin/anaconda3/envs/sfusd-project/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
print(df1)
df1 = df1.interpolate()
df2 = df1.resample('30T').mean()
df2 = df2.interpolate()
df2_data = df2.values


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df2_data)
df2_data = scaler.transform(df2_data)

for i,name in enumerate(df2.columns):
    df2[name] = df2_data[:,i]

                     Global_active_power  Global_reactive_power  Voltage  \
datetime                                                                   
2006-12-16 17:24:00                4.216                  0.418   234.84   
2006-12-16 17:25:00                5.360                  0.436   233.63   
2006-12-16 17:26:00                5.374                  0.498   233.29   
2006-12-16 17:27:00                5.388                  0.502   233.74   
2006-12-16 17:28:00                3.666                  0.528   235.68   
...                                  ...                    ...      ...   
2010-11-26 20:58:00                0.946                  0.000   240.43   
2010-11-26 20:59:00                0.944                  0.000   240.00   
2010-11-26 21:00:00                0.938                  0.000   239.82   
2010-11-26 21:01:00                0.934                  0.000   239.70   
2010-11-26 21:02:00                0.932                  0.000   239.55   

           

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.iloc[:,:-3].values, df2.iloc[:,-3:].values, test_size=0.05,shuffle=False)
data = tf.cast(X_train,tf.float32)
targets = tf.cast(y_train,tf.float32)

sample_length = 24
input_dataset = timeseries_dataset_from_array(data,None, sequence_length=sample_length,batch_size=256, sequence_stride=sample_length)
target_dataset = timeseries_dataset_from_array(targets, None, sequence_length=6,batch_size=256, sequence_stride=sample_length)

2024-08-05 23:01:02.440660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
from transformers import Transformer
xformer = Transformer(3,num_layers=4,num_heads=4,dff=256,d_model=32)
pred = xformer(tf.random.normal((1,24,4)),tf.random.normal((1,6,3)),False)
# xformer.load_weights("xformer.h5")
print(pred.shape)
xformer.summary()

ModuleNotFoundError: No module named 'transformers'

In [ ]:
from tensorflow.keras.layers import GRU,Dense,Input,TimeDistributed,RepeatVector
from tensorflow.keras.models import Model

in1 = Input((24,4))
gru1 = GRU(128)(in1)
gru2 = RepeatVector(6)(gru1)
gru2 = GRU(128,return_sequences=True)(gru2)
gru2 = TimeDistributed(Dense(3))(gru2)

model = Model(in1,gru2)
model.summary()

In [ ]:
d_model = 32
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,epsilon=1e-9)
optimizer2 = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.98,epsilon=1e-9)

len_data = sum(1 for _ in input_dataset)


In [ ]:
for e in range(200):
    for i,batch in enumerate(zip(input_dataset, target_dataset)):

        if i<len_data-1:
            in1, tar = batch
            
            tar_inp = tf.ones((256,1,3))*-0.1
            tar_inp = tf.concat([tar_inp,tar[:, :-1]],axis=1)
            tar_real = tar[:, 1:]

            with tf.GradientTape() as tape:
                predictions = xformer(in1, tar_inp,True)
                loss = tf.reduce_mean(tf.square(tar_real-predictions[:,:-1]))
            gradients = tape.gradient(loss, xformer.trainable_variables)
            optimizer.apply_gradients(zip(gradients, xformer.trainable_variables))
            
            with tf.GradientTape() as tape:
                pred2 = model(in1)
                loss2 = tf.reduce_mean(tf.square(tar-pred2))
            gradients = tape.gradient(loss2, model.trainable_variables)
            optimizer2.apply_gradients(zip(gradients, model.trainable_variables))
                
    print(f"epoch {e}| Transformer:{loss.numpy()} | GRU:{loss2.numpy()}")

In [ ]:
def predict(inp):
    
    inp = inp.reshape(-1,24,4)
#     inp = tf.reshape(inp,(-1,24,4))
    n_batch = inp.shape[0]
    start = np.ones((n_batch,1,3))*-0.1
    
    for i in range(6):
        
        pred = xformer(inp,start,False)
        pred = np.expand_dims(pred[:,-1,:],0)
        start = np.concatenate([start,pred],axis=1)
#         print(start.shape)
        
    return start[:,1:]


x_test = timeseries_dataset_from_array(X_test,None, sequence_length=sample_length,batch_size=1, sequence_stride=sample_length)
y_test = timeseries_dataset_from_array(y_test, None, sequence_length=6,batch_size=1, sequence_stride=sample_length)
x_test = list(x_test.as_numpy_iterator())
y_test = list(y_test.as_numpy_iterator())
x_test2 = np.asarray(x_test).reshape(-1,24,4)
y_test2 = np.asarray(y_test).reshape(-1,6,3)
preds_xform = [predict(x) for x in x_test2]

In [ ]:
err_xform = np.mean([(y_true-y_pred)**2 for y_true,y_pred in zip(y_test2,preds_xform)])

pred_gru = model(x_test2)
err_gru = np.mean((pred_gru-y_test2)**2)

err_xform,err_gru